# Cleaning and EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
sys.path.insert(0, '/Users/coconut/Desktop/zach/GA/Submissions/')
import eda

In [3]:
from matplotlib import rcParams
rcParams['figure.figsize'] = (12.0, 9.0)

In [4]:
# Read dataset
test = pd.read_csv('../datasets/test.csv')
train = pd.read_csv('../datasets/clean/train_clean.csv')

In [5]:
train.shape

(2049, 231)

In [6]:
test.shape

(878, 80)

___

In [7]:
to_drop = ['Lot Area', '1st Flr SF', 'Garage Yr Blt', 'TotRms AbvGrd', 'Garage Cars', 'Year Built', 'Year Remod/Add']
test.drop(columns=to_drop, inplace=True)

## Investigate null values

In [8]:
def peek_column(col_title):
    print('Unique Values:')
    print(test[col_title].unique())
    print()
    print('Value Counts:')
    print(test[col_title].value_counts())

In [9]:
def peek_columns(cols):
    for col in cols:
        peek_column(col)
        print()

## Convert ordinal features to numerical

In [10]:
map1 = {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0}
map2 = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}

In [11]:
ordinal_mapping = {
    'Exter Cond': map1,
    'Heating QC': map1,
    'Kitchen Qual': map1,
    'Bsmt Qual': map2,
    'Bsmt Cond': map2,
    'Garage Qual': map2
}

In [12]:
# Credit to Kovacs for this approach
for col, mapping in ordinal_mapping.items():
    test[col] = test[col].map(mapping)

---
## Convert Type

In [13]:
# Ensure no nulls
test['MS SubClass'].fillna(0, inplace=True)

# Make string then convert to dummies
test['MS SubClass'] = test['MS SubClass'].astype(str)

test = pd.get_dummies(test, columns=['MS SubClass']).drop(columns='MS SubClass_20')

## Pool-Related Features

In [14]:
# Engineer 'Has Pool' feature
test['Has Pool'] = test['Pool Area'].map(lambda x: 1 if x >= 1 else 0)
test.drop(columns = ['Pool QC', 'Pool Area'], inplace=True)

In [15]:
test.shape

(878, 85)

## Misc Feature

In [16]:
# Shed is the only value with high enough frequency to contribute to the model
test['Has Shed'] = test['Misc Feature'].map(lambda x: 1 if x=='Shed' else 0)
test.drop(columns = ['Misc Feature'], inplace=True)

In [17]:
test.shape

(878, 85)

## Alley

In [18]:
# Add 'Has Alley' and 'Alley Pave' features
test['Has Alley'] = test['Alley'].map(lambda x: 1 if (x == 'Pave') | (x == 'Grvl') else 0)
test['Alley Pave'] = test['Has Alley'].map({1:1, 0:0})
test.drop(columns = ['Alley'], inplace=True)

In [19]:
test.shape

(878, 86)

## Fence

In [20]:
# Dummify Fence feature
test = pd.get_dummies(test, columns=['Fence']).drop(columns='Fence_MnWw')

In [21]:
test.shape

(878, 88)

## Fireplace Qu

In [22]:
# Dummify Fireplace Qu feature
test = pd.get_dummies(test, columns=['Fireplace Qu']).drop(columns='Fireplace Qu_Ex')

In [23]:
test.shape

(878, 91)

## Lot frontage

In [24]:
# Set Lot Frontage = 0 for Lot Config = inside, corner and cul-de-sac
na_lot_frontage = test['Lot Frontage'].isna()
lot_config_to_drop = test['Lot Config'].map(lambda x: 1 if x in ['CulDSac', 'Corner', 'Inside'] else 0)
test.loc[na_lot_frontage & lot_config_to_drop, 'Lot Frontage'] = 0

In [25]:
# Set the remaining missing Lot Frontage values equal to the mean
avg_frontage = test['Lot Frontage'].mean()
test.loc[na_lot_frontage, 'Lot Frontage'] = avg_frontage

## Convert features to binary

In [26]:
# Identify binary columns
for col in test:
    if (len(test[col].unique()) == 2) & (type(test[col][0]) is str):
        print(col)
        print(test[col].unique())

Street
['Pave' 'Grvl']
Utilities
['AllPub' 'NoSewr']
Central Air
['N' 'Y']


In [27]:
# Central Air
test['Central Air'] = test['Central Air'].map({'Y':1, 'N':0})

In [28]:
# Street feature has all 2049 values. Engineer new column to indicate paved or gravel street
test['Street Pave'] = test['Street'].map({'Pave':1, 'Grvl':0})
test.drop(columns = ['Street'], inplace=True)

In [29]:
test.shape

(878, 91)

## Convert remaining categorical features to dummies

In [30]:
# Aggregate remaining nulls
nulls = eda.investigate_null(test)

In [31]:
# Extract categorical columns
col_types = {col:type(test[col][0]) for col in nulls['column']}
str_cols = [col for col,t in col_types.items() if t is str]

In [32]:
# Dummify
test = pd.get_dummies(test, columns=str_cols)

In [33]:
# Impute 0 to all remaining missing values
test.fillna(0, inplace=True)

In [34]:
# No null values left
eda.investigate_null(test)

,column,null_count


## Converting nominal and ordinal features

In [35]:
# Isolate columns of type string
col_types = {col:type(test[col][0]) for col in test.columns}
str_cols = [col for col,t in col_types.items() if t is str]

In [36]:
test = pd.get_dummies(test, columns=str_cols).reset_index(drop=True)

## Ensure train and test datasets are of same form

In [37]:
len(test.columns)

250

In [38]:
len(train.columns)

231

In [39]:
# Add salvage and severely damaged columns to new data if not already there
if 'Functional_Sal' not in test.columns:
    test['Functional_Sal'] = 0
if 'Functional_Sev' not in test.columns:
    test['Functional_Sev'] = 0

**Train data**

In [40]:
# Identify columns to drop in training dataset
to_drop = []
for col in train.columns:
    if col not in test.columns:
        to_drop.append(col)

In [41]:
to_drop

['SalePrice',
 'MS SubClass_150',
 'Utilities_NoSeWa',
 'Neighborhood_GrnHill',
 'Neighborhood_Landmrk',
 'Condition 2_PosN',
 'Condition 2_RRAe',
 'Condition 2_RRAn',
 'Condition 2_RRNn',
 'Roof Matl_Membran',
 'Exterior 1st_CBlock',
 'Exterior 1st_ImStucc',
 'Exterior 1st_Stone',
 'Exterior 2nd_Stone',
 'Heating_OthW',
 'Heating_Wall']

In [42]:
# Keep SalePrice
to_drop = to_drop[1:]

In [43]:
train.drop(columns = to_drop, inplace = True)

**Same process for new data**

In [44]:
# Identify columns to drop in testing dataset
to_drop = []
for col in test.columns:
    if col not in train.columns:
        to_drop.append(col)

In [45]:
# Keep Id column for scoring
to_drop.pop(0)

'Id'

In [46]:
# Peek columns to drop
test[to_drop].sum()

PID                     628899719478
Garage Finish_Fin                218
Garage Cond_Gd                     3
Garage Type_CarPort                4
Bsmt Exposure_Mn                  76
BsmtFin Type 1_LwQ                52
BsmtFin Type 2_GLQ                11
Mas Vnr Type_BrkCmn               12
Mas Vnr Type_CBlock                1
Electrical_FuseP                   1
Lot Shape_IR1                    286
Land Contour_Bnk                  36
Utilities_AllPub                 877
Lot Config_Corner                163
Land Slope_Gtl                   835
Neighborhood_Blmngtn               6
Condition 1_Artery                22
Bldg Type_1Fam                   724
House Style_1.5Fin                96
Roof Style_Flat                    7
Roof Matl_CompShg                861
Roof Matl_Metal                    1
Roof Matl_Roll                     1
Exterior 1st_AsbShng              11
Exterior 1st_PreCast               1
Exterior 2nd_AsbShng              10
Exterior 2nd_Other                 1
E

In [47]:
test.drop(columns = to_drop, inplace = True)

In [48]:
len(test.columns)

216

In [49]:
len(train.columns)

216

In [50]:
# Order dataframe columns consistently
train.sort_index(axis=1, inplace=True)
test.sort_index(axis=1, inplace=True)

## Save clean dataset

In [51]:
# Train
train.to_csv('../datasets/clean/train_clean.csv', index=False)

In [52]:
# Test
test.to_csv('../datasets/clean/test_clean.csv', index=False)